First install dependencies:

pip install openmeteo-requests
pip install requests-cache retry-requests numpy pandas

In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
responses = []
df = pd.DataFrame()

In [3]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
	"latitude": [40.4165,51.5085,48.8534,52.5244,50.8505,41.0138 ],
	"longitude": [3.7026,-0.1257,2.3488,13.4105,4.3488,28.9497],
	"start_date": "2020-01-01",
	"end_date": "2025-01-01",
	"daily": ["weather_code",
           "temperature_2m_mean",
           "temperature_2m_max",
           "temperature_2m_min",
           "precipitation_sum",
           "rain_sum",
           "snowfall_sum",
           "precipitation_hours",
           "wind_speed_10m_max",
           "wind_gusts_10m_max",
           "wind_direction_10m_dominant",
           "shortwave_radiation_sum"]
}

coordinates = {'madrid':{'latitud':'40,4165', 'longitude':'-3,7026'},
               'london':{'latitud':'51,5085', 'longitude':'-0,1257'},
               'paris':{'latitud':'48,8534', 'longitude':'2,3488'},
               'berlin':{'latitud':'52,5244', 'longitude':'13,4105'},
               'brussels':{'latitud':'50,8505', 'longitude':'4,3488'},
               'istanbul':{'latitud':'41,0138', 'longitude':'28,9497'}
}

In [4]:
def get_weather_data():
    responses = openmeteo.weather_api(url, params=params)
    return responses

In [5]:
responses = get_weather_data()

In [6]:
def get_weather_dataframe():
    daily_dataframe = pd.DataFrame()
    for response in responses:
        daily = response.Daily()
        daily_weather_code = daily.Variables(0).ValuesAsNumpy()
        daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
        daily_temperature_2m_max = daily.Variables(2).ValuesAsNumpy()
        daily_temperature_2m_min = daily.Variables(3).ValuesAsNumpy()
        daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
        daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
        daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
        daily_precipitation_hours = daily.Variables(7).ValuesAsNumpy()
        daily_wind_speed_10m_max = daily.Variables(8).ValuesAsNumpy()
        daily_wind_gusts_10m_max = daily.Variables(9).ValuesAsNumpy()
        daily_wind_direction_10m_dominant = daily.Variables(10).ValuesAsNumpy()
        daily_shortwave_radiation_sum = daily.Variables(11).ValuesAsNumpy()

        daily_data = {"date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
        )}

        daily_data["weather_code"] = daily_weather_code
        daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
        daily_data["temperature_2m_max"] = daily_temperature_2m_max
        daily_data["temperature_2m_min"] = daily_temperature_2m_min
        daily_data["precipitation_sum"] = daily_precipitation_sum
        daily_data["rain_sum"] = daily_rain_sum
        daily_data["snowfall_sum"] = daily_snowfall_sum
        daily_data["precipitation_hours"] = daily_precipitation_hours
        daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
        daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
        daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
        daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

        temp_df = pd.DataFrame(data = daily_data)
        daily_dataframe = pd.concat([daily_dataframe,temp_df])
    return daily_dataframe


In [7]:
df = get_weather_dataframe()

In [11]:
display(df)

,date,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum
0,2020-01-01 00:00:00+00:00,3.0,12.989583,13.8500,12.5500,0.000000,0.000000,0.0,0.0,16.935310,24.840000,277.352295,8.37
1,2020-01-02 00:00:00+00:00,51.0,13.664582,14.1500,13.2500,0.200000,0.200000,0.0,2.0,14.058450,19.799999,219.161346,7.14
2,2020-01-03 00:00:00+00:00,51.0,14.016665,14.2500,13.8000,0.400000,0.400000,0.0,4.0,19.353140,25.559999,253.110992,5.15
3,2020-01-04 00:00:00+00:00,3.0,13.872918,14.4500,13.4000,0.000000,0.000000,0.0,0.0,28.820242,38.519997,304.691925,8.75
4,2020-01-05 00:00:00+00:00,3.0,13.956250,14.7500,13.2500,0.000000,0.000000,0.0,0.0,36.115017,49.320000,339.703979,8.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,2024-12-28 00:00:00+00:00,51.0,7.460083,8.7955,5.1955,1.400000,1.400000,0.0,10.0,26.925497,65.520004,34.731762,4.62
1824,2024-12-29 00:00:00+00:00,53.0,7.226750,7.6955,6.7955,3.299999,3.299999,0.0,17.0,23.766136,52.560001,24.173382,1.44
1825,2024-12-30 00:00:00+00:00,3.0,6.597583,8.2955,4.9455,0.000000,0.000000,0.0,0.0,19.189215,43.199997,12.839953,4.93
1826,2024-12-31 00:00:00+00:00,3.0,6.632999,10.3955,4.2455,0.000000,0.000000,0.0,0.0,9.403169,18.359999,19.524181,7.55


In [10]:
df.to_csv('weather_output.csv')